In [1]:
!pip install econml

In [2]:
#Required Libraries
from econml.metalearners import XLearner
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as st
import matplotlib.pyplot as plt

In [3]:
class metrics:
    
    def pehe(self,effect_true, effect_pred):
        """
        Precision in Estimating the Heterogeneous Treatment Effect (PEHE)
        :param effect_true: true treatment effect value
        :param effect_pred: predicted treatment effect value
        :return: PEHE
        """
        return np.abs(np.mean(effect_pred) - np.mean(effect_true))

    def abs_ate(self,effect_true, effect_pred):
        """
        Absolute error for the Average Treatment Effect (ATE)
        :param effect_true: true treatment effect value
        :param effect_pred: predicted treatment effect value
        :return: absolute error on ATE
        """
        return np.sqrt(np.mean((effect_true - effect_pred)**2))
    @staticmethod
    def abs_att(effect_pred, yf, t, e):
        """
        Absolute error for the Average Treatment Effect on the Treated
        :param effect_pred: predicted treatment effect value
        :param yf: factual (observed) outcome
        :param t: treatment status (treated/control)
        :param e: whether belongs to the experimental group
        :return: absolute error on ATT
        """
        att_true = np.mean(yf[t > 0]) - np.mean(yf[(1 - t + e) > 1])
        att_pred = np.mean(effect_pred[(t + e) > 1])

        return np.abs(att_pred - att_true)
    @staticmethod
    def policy_risk(effect_pred, yf, t, e):
        """
        Computes the risk of the policy defined by predicted effect
        :param effect_pred: predicted treatment effect value
        :param yf: factual (observed) outcome
        :param t: treatment status (treated/control)
        :param e: whether belongs to the experimental group
        :return: policy risk
        """
        # Consider only the cases for which we have experimental data (i.e., e > 0)
        t_e = t[e > 0]
        yf_e = yf[e > 0]
        effect_pred_e = effect_pred[e > 0]

        if np.any(np.isnan(effect_pred_e)):
            return np.nan

        policy = effect_pred_e > 0.0
        treat_overlap = (policy == t_e) * (t_e > 0)
        control_overlap = (policy == t_e) * (t_e < 1)

        if np.sum(treat_overlap) == 0:
            treat_value = 0
        else:
            treat_value = np.mean(yf_e[treat_overlap])

        if np.sum(control_overlap) == 0:
            control_value = 0
        else:
            control_value = np.mean(yf_e[control_overlap])

        pit = np.mean(policy)
        policy_value = pit * treat_value + (1.0 - pit) * control_value

        return 1.0 - policy_value


In [4]:
metrics = metrics()


### Data exploration, Preprocessing & Modelling

In [5]:
# loading jobs Dataset
df = np.load('../input/datatest/jobs.npz')
"""x = Feature Variable, t --> Treatment, y --> Outcome Variable (Factual)
   e --> experimental or observational data"""
for f in df.files:
  print(f'{f}: {df[f].shape}')
jx , jt , jy, je = df['x'], df['t'], df['y'], df['e']
dfX,dfT,dfY,dfE =  pd.DataFrame(df['x']),pd.DataFrame(df['t']),pd.DataFrame(df['y']),pd.DataFrame(df['e'])
print(dfX.info())


In [6]:
dfX.describe().T

In [7]:
dfX.boxplot()

In [8]:
sns.pairplot(data=dfX)

In [9]:

fig, axs = plt.subplots(1,4, figsize=(16, 4))
axs[0].hist(dfX, bins=20)
axs[1].hist(dfT, bins=20)
axs[2].hist(jy, bins=20)
axs[3].hist(je, bins=20)
plt.show()

In [10]:
plt.figure(figsize=(16, 10))
heatmap = sns.heatmap(dfX.corr(), vmin=-1, vmax=1, annot=True)
heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':12}, pad=12);

Because there appear to be no missing data or non-numerical values in jobs, there is no need for preprocessing when encoding and filling Nan rows, as there is with the IHDP dataset. Jobs, like IHDP, have a lot of outliers in the background variables, which requires a similar experiment with normalisation approach. However, we may investigate random forest regression models, which should manage any outliers internally and reduce the likelihood of our results being skewed or biassed in any way.

In [11]:
dfX.hist(bins=25,figsize=(14,10))


the background variables in Jobs seem to be unbalanced

In [12]:
fig, axs = plt.subplots(1, 2, figsize=(16, 4))
limit = 20
axs[0].scatter(jx[:, 0].reshape(-1, 1)[jt == 1][:limit]
               , jy[jt == 1][:limit], label = "Treated")
axs[0].scatter(jx[:, 0].reshape(-1, 1)[jt == 0][:limit]
               , jy[jt == 0][:limit], label = "Control")
axs[1].scatter(jx[:, 1].reshape(-1, 1)[jt == 1][:limit]
               , jy[jt == 1][:limit], label = "Treated")
axs[1].scatter(jx[:, 1].reshape(-1, 1)[jt == 0][:limit]
               , jy[jt == 0][:limit], label = "Control")
axs[0].legend(ncol=2)
axs[1].legend(ncol=2)
plt.show()

In contrast to IHDP, jobs outcomes are recorded as binary variables, therefore scatter points are only plotted on 0 and 1. Given the four graphs above, it's far more difficult to spot noticeable effects; yet, given the background variables we've chosen to depict, this could just be a coincidence.

In [13]:
fig, axs = plt.subplots(1, 2, figsize=(16, 4))
limit = 20
axs[0].scatter(jx[:, 2].reshape(-1, 1)[jt == 1][:limit],
               jy[jt == 1][:limit], label = "Treated")
axs[0].scatter(jx[:, 2].reshape(-1, 1)[jt == 0][:limit]
               , jy[jt == 0][:limit], label = "Control")
axs[1].scatter(jx[:, 3].reshape(-1, 1)[jt == 1][:limit]
               , jy[jt == 1][:limit], label = "Treated")
axs[1].scatter(jx[:, 3].reshape(-1, 1)[jt == 0][:limit],
               jy[jt == 0][:limit], label = "Control")
axs[0].legend(ncol=2)
axs[1].legend(ncol=2)
plt.show()

In [14]:
bins=20
plt.hist(jt, bins=bins, color='hotpink')
plt.title("Treatment and Control Distribution")
plt.show()

The graphs above demonstrate why we need to apply X-learner. In both datasets, there is an obvious imbalance in favour of the treatment and control groups; hopefully, X-learner will be able to account for this when calculating our CATE value.

### Data Modelling and Standardizing


In [15]:
jx_train, jx_test, jt_train, jt_test, jy_train, jy_test, je_train, je_test = train_test_split(jx, jt, jy, je, test_size=0.2)

## Training Models - Standard Models

### Random Forest Regressor 

In [16]:
# Simple Random forest trained using the Jobs features + treatment
rf_jobs = RandomForestRegressor() 
Conc_train_j = np.concatenate([jx_train, jt_train], axis=1)
rf_jobs.fit(Conc_train_j, jy_train.flatten())

# predict Y0 and Y1 given t=0 and t=1 respectively for both datasets
# t=0 - Jobs
Concatenated_XT_zeros_Jobs = np.concatenate([jx_test, np.zeros_like(jt_test)], axis=1)
j_y0 = rf_jobs.predict(Concatenated_XT_zeros_Jobs)
# t=1 - Jobs
Concatenated_XT_ones_Jobs = np.concatenate([jx_test, np.ones_like(jt_test)], axis=1)
j_y1 = rf_jobs.predict(Concatenated_XT_ones_Jobs)

estimated_eff = j_y1 - j_y0
ATT_Jobs = metrics.abs_att(estimated_eff, jy_test.flatten()
                           , jt_test.flatten(), je_test.flatten())
Rpol = metrics.policy_risk(estimated_eff, jy_test.flatten()
                           , jt_test.flatten(), je_test.flatten())
print( "RF ATT, RF RISK POLICY :", ATT_Jobs, Rpol )

As discussed in the report, the Jobs dataset wil be evaluated using ATT and Rpol

In [17]:
temp_XJ = pd.DataFrame(jx_train)
temp_XJ_t = pd.DataFrame(jx_test)
#temp_X_Jobs.head()
#[temp_X_Jobs[cols].unique() for cols in temp_X_Jobs]
temp_XJ.head()

In [18]:
# Jobs
# Scale columns 0,1,6,7,8,9,10,11,12,15 (all non binary)
temp_XJ.iloc[:, [0,1,6,7,8,9,10,11,12,15]] = StandardScaler().fit_transform(temp_XJ.iloc[:, [0,1,6,7,8,9,10,11,12,15]])
temp_XJ_t.iloc[:, [0,1,6,7,8,9,10,11,12,15]] = StandardScaler().fit_transform(temp_XJ_t.iloc[:, [0,1,6,7,8,9,10,11,12,15]]) 
Jobs_xtrain_stan = temp_XJ.to_numpy()
Jobs_xtest_stan = temp_XJ_t.to_numpy()
temp_XJ.head()

### Random Forest Regressor - Optimized

10 fold cross validation Grid Search in combination with Random search

#### Grid Search

In [19]:
Benchmark_Jobs = RandomForestRegressor()
# Our parameter Grid uses values centered around the parameters found from the random search
param_grid = {
    'bootstrap': [True],
    'max_depth': [10, 20, 30, 40, 50],
    'max_features': ['auto', 'sqrt'],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [4, 5, 6],
    'n_estimators': [600, 800, 1000]
}

GridSearch_RF_Jobs = GridSearchCV(estimator = Benchmark_Jobs, param_grid = param_grid, cv = 10, n_jobs = -1, verbose = 2)
GridSearch_RF_Jobs.fit(Conc_train_j, jy_train.flatten())
GridSearch_RF_Jobs.best_params_

#### Metric Evaluation

In [20]:
tuned_rf = RandomForestRegressor(bootstrap=True, max_depth=90, max_features='sqrt', min_samples_leaf=3, min_samples_split=6, n_estimators=800)
tuned_rf.fit(Conc_train_j, jy_train.flatten())
# t=0 
Conc_zeros_j = np.concatenate([jx_test, np.zeros_like(jt_test)], axis=1)
Y0_Jobs = tuned_rf.predict(Conc_zeros_j)
# t=1 
Conc_ones_j = np.concatenate([jx_test, np.ones_like(jt_test)], axis=1)
Y1_Jobs = tuned_rf.predict(Conc_ones_j)
#ITEs for Jobs dataset
tuned_estimated_eff = Y1_Jobs - Y0_Jobs
ATT_Jobs_Optimized = metrics.abs_att(tuned_estimated_eff, jy_test.flatten(), jt_test.flatten(), je_test.flatten())
Rpol_Optimized = metrics.policy_risk(tuned_estimated_eff, jy_test.flatten(), jt_test.flatten(), je_test.flatten())
print('RF After hyperparameter tuning ATT, Risk Policy :', ATT_Jobs_Optimized, Rpol_Optimized)

From the metric table above, we can see no improvement on the accuracy of the effect predictions on the treated group; this being said we have a slight improvement in the models risk policy metric. The effect of the optimized model although isn't significant, could be effective for proving NSWP training affects a persons employability

### Feature Importances 

In [21]:
# Feature Importances for the Jobs dataset using random forest regression  
tuned_rf.fit(Conc_train_j, jy_train.flatten())
features_Jobs = list(Conc_train_j)
importances = tuned_rf.feature_importances_
std_Jobs = np.std([tree.feature_importances_ for tree in tuned_rf.estimators_], axis=0)
indices_Jobs_RF = np.argsort(importances)[::-1]
print("")
print("Feature Importances - Random Forest")
for f in range(Conc_train_j.shape[1]):
    print("%d. %s (%f)" % (f + 1, "Feature Column: "+ str(indices_Jobs_RF[f]), importances[indices_Jobs_RF[f]]))

In [22]:
fig, axs = plt.subplots(1, 2, figsize=(16, 4))
axs[0].bar(range(Conc_train_j.shape[1]), importances[indices_Jobs_RF], color="r", yerr=std_Jobs[indices_Jobs_RF], align="center")
axs[0].set_xticks(range(Conc_train_j.shape[1]), indices_Jobs_RF)
axs[0].set_xlim([-1, Conc_train_j.shape[1]])
axs[0].set_ylim([0, None])
axs[0].set_title("Feature Importances - Optimized Random Forest", fontsize=12, fontweight="bold")
plt.show()

## Propensity Score Re-weighting

In [23]:
simple_clf = RandomForestClassifier()
simple_clf.fit(jx_train, np.squeeze(jt_train))
# These will act as our ptx values
ex_Jobs = simple_clf.predict_proba(jx_train).T[1].T + 0.0001

    
$$w_i = \frac{t_i}{e(x_i)} + \frac{1-t_i}{1-e(x_i)}$$



In [24]:
import math
def Calc_Weights(ti, ex):
    return (ti / ex) + ((1-ti) / (1-ex))

In [25]:
Sample_Weights_Jobs = Calc_Weights(np.squeeze(jt_train), ex_Jobs)

Now that we have our sample weights, we can train our regressors using the weights

In [26]:
# Training variables for random forest 
Concatenated_XT_train_Jobs = np.concatenate([jx_train, jt_train], axis=1)
RandomForest_Jobs_IPSW = RandomForestRegressor()
# Trained regressors
RandomForest_Jobs_IPSW.fit(Concatenated_XT_train_Jobs, jy_train.flatten(), sample_weight=Sample_Weights_Jobs)

#### Random forest Grid Search

In [27]:
# Our parameter Grid uses values centered around the parameters found from the random search
param_grid = {
    'bootstrap': [True],
    'max_depth': [40, 50, 60],
    'max_features': ['auto', 'sqrt'],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [5, 7, 9],
    'n_estimators': [400, 600, 800]
}

GridSearch_RF_Jobs_IPSW = GridSearchCV(estimator = RandomForest_Jobs_IPSW, param_grid = param_grid, cv = 10, n_jobs = -1, verbose = 2)
GridSearch_RF_Jobs_IPSW.fit(Concatenated_XT_train_Jobs, jy_train.flatten(), sample_weight=Sample_Weights_Jobs)
GridSearch_RF_Jobs_IPSW.best_params_

#### Metric Evaluation

In [28]:
RF_IPSW_Jobs = RandomForestRegressor(bootstrap=True, max_depth=60, max_features='sqrt', min_samples_leaf=4, min_samples_split=9, n_estimators=600)
RF_IPSW_Jobs.fit(Conc_train_j, jy_train.flatten(), sample_weight=Sample_Weights_Jobs)
# t=0 - Jobs
Concatenated_XT_zeros_Jobs = np.concatenate([jx_test, np.zeros_like(jt_test)], axis=1)
Y0_Jobs_IPSW_RF = RF_IPSW_Jobs.predict(Concatenated_XT_zeros_Jobs)
# t=1 - Jobs
Concatenated_XT_ones_Jobs = np.concatenate([jx_test, np.ones_like(jt_test)], axis=1)
Y1_Jobs_IPSW_RF = RF_IPSW_Jobs.predict(Concatenated_XT_ones_Jobs)
#ITEs for Random Forest Model Jobs IPSW
Effect_Estimates_Jobs_RF_IPSW = Y1_Jobs_IPSW_RF - Y0_Jobs_IPSW_RF
# Metric calculations for IPSW
ATT_Jobs_RF_IPSW = metrics.abs_att(Effect_Estimates_Jobs_RF_IPSW, jy_test.flatten(), jt_test.flatten(), je_test.flatten())
Rpol_RF_IPSW     = metrics.policy_risk(Effect_Estimates_Jobs_RF_IPSW, jy_test.flatten(), jt_test.flatten(), je_test.flatten())
print('IPSW Random Forest ATT, Risk Policy :',ATT_Jobs_RF_IPSW ,Rpol_RF_IPSW)

The propensity score re-weighting has no noticeable impact on prediction performance. The error readings for the optimised linear regressor are still the lowest. Now we can see how the feature importances were influenced by the propensity score weighting. We see a rise in the significance of the treatment feature.

### Feature Importances for IPSW 

In [29]:
from matplotlib import pyplot

# Feature Importances for the Jobs dataset using random forest regression  
features_Jobs = list(Conc_train_j)
importances = RF_IPSW_Jobs.feature_importances_
std_Jobs = np.std([tree.feature_importances_ for tree in RF_IPSW_Jobs.estimators_], axis=0)
indices_Jobs_RF = np.argsort(importances)[::-1]
print("")
print("Feature Importances - RF IPSW")
for f in range(Conc_train_j.shape[1]):
    print("%d. %s (%f)" % (f + 1, "Feature Column: "+ str(indices_Jobs_RF[f]), importances[indices_Jobs_RF[f]]))
pyplot.bar([x for x in range(len(importances))], importances)
pyplot.title("Feature Importance for Jobs dataset using RF")
pyplot.ylabel("Feature Score")
pyplot.xlabel("Feature Number")
pyplot.show()
    

## Task 5 CATE Estimators

Due to its performance, we will be using random forest as the base learners for XL

In [30]:
xl_Jobs = XLearner(models=RandomForestRegressor(), propensity_model=RandomForestClassifier())
xl_Jobs.fit(jy_train, jt_train.flatten(), X=jx_train)
xl_te_test_Jobs = xl_Jobs.effect(jx_test)

Calculating our ATE and PEHE metrics 

In [31]:
xl_att_test = metrics.abs_att(xl_te_test_Jobs, jy_test.flatten(), jt_test.flatten(), je_test.flatten())
xl_rpol = metrics.policy_risk(xl_te_test_Jobs.flatten(), jy_test.flatten(), jt_test.flatten(), je_test.flatten())

In [32]:
# Metrics for Jobs Dataset
results_ = []
results_.append(['Random Forest', ATT_Jobs, Rpol])
results_.append(['Hyperparameter tuned Random Forest', ATT_Jobs_Optimized, Rpol_Optimized])
results_.append(['IPSW Random Forest',ATT_Jobs_RF_IPSW ,Rpol_RF_IPSW])
results_.append(['X-Learner',xl_att_test ,xl_rpol])
cols_ = ['Methodology', 'ATT test', 'Risk Policy']
df_Second = pd.DataFrame(results_, columns=cols_)
df_Second

It seems that the best performing model for this optimised dataset is the standard random forest; having not only the most accurate average treatment effect on the treated, but also a fairly low risk policy. 